In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [2]:
from utils import get_cuda_info

get_cuda_info()

PyTorch version: 2.5.1+cu118
**********
_CUDA version: 
CUDA version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0

**********
CUDNN version: 90100
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4070 Ti SUPER


## Zdobycie danych

In [3]:
from utils import load_data, convert_landmarks_to_eye_movements

all_data, all_labels = load_data('silesian_deception')

In [14]:
converted_data, blinks = convert_landmarks_to_eye_movements(all_data)

## Ograniczenie do pierwszych N klatek

In [15]:
import numpy as np
blinks = np.array([d[:1000] for d in blinks], dtype=object)

## Preprocessing danych

In [18]:
from utils import preprocess_data

X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(blinks, all_labels, binarize_labels=False)

In [19]:
print(X_train.shape)
print(y_train.shape)

torch.Size([700, 1000])
torch.Size([700])


In [20]:
from utils import get_class_distribution

get_class_distribution(all_labels)

===> Class distribution <===
0: 301
1: 699
=============><=============


# MODEL TORCH

## Zbudowanie modelu ekstrakcji cech

In [21]:
import torch
import torch.nn as nn

In [ ]:
class BlinkingTransformerClassifier(nn.Module):
    def __init__(self, d_model=64, num_heads=4, num_layers=2):
        super().__init__()
        self.input_proj = nn.Linear(1, d_model)  # 1 feature: blink signal

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dropout=0.1)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.Linear(d_model, 1)  # binary output
        )

    def forward(self, x):  # x shape: [batch, time]
        x = x.unsqueeze(-1)         # -> [batch, time, 1]
        x = self.input_proj(x)      # -> [batch, time, d_model]
        x = x.permute(1, 0, 2)      # -> [time, batch, d_model] for Transformer
        x = self.encoder(x)         # -> [time, batch, d_model]
        x = x.mean(dim=0)           # average over time: [batch, d_model]
        out = self.classifier(x)    # -> [batch, 1]
        return out

In [30]:
from torch.optim import Adam

model = BlinkingTransformerClassifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pos_weight = torch.tensor([(len(y_train) - y_train.sum()) / y_train.sum()]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = Adam(model.parameters(), lr=1e-4)

C:\Users\Bartosz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


## Trening modelu

In [31]:
from torch.utils.tensorboard import SummaryWriter

RUNS_FOLDER_PATH = os.path.abspath('runs')
writer_path = os.path.join('runs', model.__class__.__qualname__, 'blinking')
writer = SummaryWriter(writer_path)

In [32]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [33]:
from utils.model_functions import train_torch_model_binary

train_torch_model_binary(model, criterion, optimizer, X_train, y_train, X_val, y_val, writer=writer, batch_size=128, unbalanced=True, show_prediction_stats=False)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

## Ewaluacja modelu

In [26]:
from utils.model_functions import eval_torch_model_binary

eval_torch_model_binary(model, criterion, X_test, y_test)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                                             VALIDATION                                             
----------------------------------------------------------------------------------------------------
Loss        : 1.3861
Accuracy    : 0.5208
Precision   : 0.2604
Recall      : 0.5000
F1 Score    : 0.3425
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                          
TP Rate     : 1.0000                                    FP Rate     : 1.0000



# TODYNET

### Przygotowanie danych

In [15]:
X_train_np = X_train.numpy()
X_val_np = X_val.numpy()
X_test_np = X_test.numpy()
y_train_np = y_train.numpy()
y_val_np = y_val.numpy()
y_test_np = y_test.numpy()

In [16]:
TodyNet_DATA_PATH = os.path.join("..", "..", "src", "external", "TodyNet", "data", "UCR", "MIAMI_DECEPTION_EYE_MOVEMENT")

os.makedirs(TodyNet_DATA_PATH, exist_ok=True)

In [17]:
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32).unsqueeze(1)  # adding channel dimension
X_val_tensor = torch.tensor(X_val_np, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32).unsqueeze(1)

# Save the data in PyTorch (.pt) format
torch.save(X_train_tensor, os.path.join(TodyNet_DATA_PATH, 'X_train.pt'))
torch.save(X_val_tensor, os.path.join(TodyNet_DATA_PATH, 'X_valid.pt'))
torch.save(X_test_tensor, os.path.join(TodyNet_DATA_PATH, 'X.pt'))

# Save the labels in PyTorch (.pt) format
torch.save(y_train, os.path.join(TodyNet_DATA_PATH, 'y_train.pt'))
torch.save(y_val, os.path.join(TodyNet_DATA_PATH, 'y_valid.pt'))
torch.save(y_test, os.path.join(TodyNet_DATA_PATH, 'y.pt'))

In [18]:
X_train_tensor.shape

torch.Size([224, 1, 1679, 2])

### Trening modelu [pool_ratio 0.8, ponieważ rozmiar danych jest zbyt duży na 0.2]

In [ ]:
# cd .\src\external\TodyNet\src\ & python train.py --dataset='MIAMI_DECEPTION_EYE_MOVEMENT' --num_layers 1 --in_dim 16 --hidden_dim 16 --out_dim 16 --pool_ratio 0.0 --kern_size "3" --groups 1